In [15]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
import sqlite3 as sqlt
import time 
from itertools import cycle
import random

In [15]:
user_agent_list = [
   #Chrome
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    #Firefox
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
]

In [137]:
def get_dados_movida(con, cursor, inserir_dados):
    url = "https://busca.movidaseminovos.com.br/filtros/class/usado?sort=11&page={}"
    indice_pagina = 1
    dados = {}
    #proxies = cycle(get_proxies())
    while True:
        #proxy = next(proxies)
        #proxy_dict = {"http": proxy, "https": proxy}
        #headers = {'User-Agent': random.choice(user_agent_list)}
        #headers = {'User-Agent':random.choice(user_agent_list)}
        #r = req.get(url.format(indice_pagina),headers=headers)
        r = req.get(url.format(indice_pagina))
        #print(r.request.headers)
        #print("\n")
        soup = bs(r.text, "lxml")
        if soup.find_all(class_ = "nm-not-found-message"):
            r.close()
            break
        for carro in soup.find_all(class_ = "nm-product-info"):
            dados["Empresa"] = "Movidas"
            texto = list(carro.stripped_strings)
            dados["Modelo"] = texto[0]
            dados["Preco"] = texto[1][3:].replace(".","").replace(",",".")
            dados["Kilometragem"] = texto[2]
            dados["Ano"] = texto[4]
            cursor.execute(inserir_dados.format(**dados))
        indice_pagina = indice_pagina + 1
        time.sleep(1.2)
        con.commit()

In [138]:
def get_dados_unidas(con, cursor, inserir_dados):
    url = "https://www.seminovosunidas.com.br/veiculos/page:{}?" + \
        "utm_source=afilio&utm_medium=display&utm_campaign=maio&utm_content" + \
        "=ron_ambos&utm_term=120x600_promocaomaio_performance_-_-"
    indice_pagina = 1
    dados = {}
    while True:
        #headers = {'User-Agent':random.choice(user_agent_list)}
        r = req.get(url.format(indice_pagina))
        soup = bs(r.text, "lxml")
        pagina_atual = list(soup.find(class_="active number").stripped_strings)[0]
        if indice_pagina >= 2 and pagina_atual == '1': 
            r.close()
            break
        lista1 = soup.find_all(class_="vehicleDescription")
        lista2 = soup.find_all(class_="valor")
        for carro, valor in zip(lista1,lista2):
            texto = list(carro.stripped_strings)
            dados["Empresa"] = "Unidas"
            dados["Modelo"] = texto[2]
            dados["Preco"] = valor.text.replace(".","").replace(",",".")
            dados["Kilometragem"] = texto[4].split(",")[1][5:]
            dados["Ano"] = texto[3][-5:-1]
            cursor.execute(inserir_dados.format(**dados))
        con.commit()
        indice_pagina += 1       

In [139]:
def get_dados_locamerica(con, cursor, inserir_dados):
    url = "https://seminovos.locamerica.com.br/veiculos.json?marca=&modelo=&cambio=&com" + \
        "bustivel=&cor=&acessorios=&por_pagina=36&per_page={}&anode=0&precode=17990" + \
        "00&precoate=13499000"
        
    indice_pagina = 1
    dados = {}
    while True:
        r = req.get(url.format(indice_pagina))
        info = r.json()['veiculos']
        site_decodificado = base64.b64decode(info)
        soup = bs(site_decodificado, "lxml")
        if not soup.find(class_="clearfix"):
            r.close()
            break
        for carro in soup.find_all(class_="item-carro"):
            texto = list(carro.stripped_strings)
            dados["Empresa"] = "Locamerica"
            preco = carro.find(class_= "preco").h4.text
            dados["Preco"] = preco.replace(".","").replace(",",".")
            dados["Modelo"] = texto[1]
            dados["Ano"] = texto[3].split("/")[0]
            kilometragem = texto[7].split()[0]
            if kilometragem.count(".") > 1: # maior que 1000 KMs
                kilometragem = kilometragem.replace(".","",1) # tira o primeiro "."
            if kilometragem == '-':
                dados["Kilometragem"] = 0
            else:
                dados["Kilometragem"] = kilometragem
            cursor.execute(inserir_dados.format(**dados))
        con.commit()
        indice_pagina += 1  
    
    
    
    

In [140]:
with sqlt.connect("seminovos.db") as con:
    cursor = con.cursor()
    criar_tabela_sql = """
    CREATE TABLE vendas 
    (  
     ID INTEGER PRIMARY KEY AUTOINCREMENT,  
     Empresa VARCHAR (20) NOT NULL,  
     Modelo  VARCHAR(40) NOT NULL,  
     Preco REAL NOT NULL,
     Kilometragem REAL NOT NULL,
     Ano NUMERIC(4,0) NOT NULL
    );  
    """

    cursor.execute(criar_tabela_sql)

    inserir_dados_sql = """
    INSERT INTO vendas (Empresa, Modelo, Preco, Kilometragem, Ano) 
    VALUES ("{Empresa}","{Modelo}", {Preco},{Kilometragem},{Ano})
    """
    inicio = time.time()
    get_dados_movida(con,cursor,inserir_dados_sql)
    get_dados_unidas(con, cursor, inserir_dados_sql)
    get_dados_locamerica(con, cursor, inserir_dados_sql)
    print(time.time() - inicio)

445.61708068847656


In [2]:
url = "https://seminovos.localiza.com/Paginas/resultado-busca.aspx?ct=4365_2002_8466_8607_8655_4389_2604_2612_8096_1734_4720_8719_3970_7267_7300_2826_8146_5758_6667_565_8167_1307_2108_7478_8875_2372_6698_8220_4777_8234_3159_8987_6018_108_4498_9040_6057_9061_6974_6744_6749_9123_7690_9185_7719_5210_6797_957_9317_9328_9332_9352_7876_2453_9362_1968_9391_9420_5454_3873_3874_4337_1987_1081_6875&st=AL_BA_CE_DF_ES_GO_MA_MG_MS_MT_PA_PB_PE_PI_PR_RJ_RN_RS_SC_SE_SP&yr=2013_2018&pc=20000_425000&fb=W_X_T_%u00d4_A_D_C_L_1_8_F_M_U_O_R_G_B&md=000192_000097_000148_000147_000136_000119_000137_000120_000729_001061_000132_000632_000699_000122_000041_000286_000772_000719_000180_000181_000179_000715_000736_000854_000748_001076_000334_000250_000333_000330_000332_000369_000424_000545_000511_000488_000408_000418_000510_000513_000391_000211_000431_001083_000325_000326_000344_000456_000451_000455_000458_000505_000506_000502_000805_000426_000427_000623_000726_000718_000119_000132_000694_000788_000828_000005_000484_000478_000481_000476_000473_000477_001017_001016_000356_000357_000319_000317_000297_000298_000354_000123_000780_000859_000138_001006_000614_000867_000858_000675_000139_000165_000171_000174_000039_000020_000022_000019_000747_000320_000365_000311_000355_000312_000313_000322_000342_000353_000362_000314_000699_000696_000114_000779_000781_000755_000807_000806_001084_000529_001063_001098_000667_001036_001077_000133_000143_000658_000705_000707_000047"
r = req.get(url)
print(r.request.headers)
soup = bs(r.text,"lxml")
valores = soup.find_all(class_="busca-right-container")
carros = soup.find_all(class_="busca-left-container")
for carro, valor in zip(carros, valores): 
    dados = {}
    dados["Preco"] = list(valor.stripped_strings)[0][3:].replace(".","")
    texto = list(carro.stripped_strings)
    dados["Modelo"] = texto[0]
    dados["Ano"] = texto[1].split("/")[0]
    dados["Kilometragem"] = texto[2]
    print(dados)
    print("##################################")
print("pagina 2")
print("###################################")
payload = {"ctl00$ctl42$g_f221d036_75d3_4ee2_893d_0d7b40180245$hdnPaginaAtual": 2}
r = req.post(url, data = payload)
print(r.request.headers)
soup = bs(r.text,"lxml")
valores = soup.find_all(class_="busca-right-container")
carros = soup.find_all(class_="busca-left-container")
for carro, valor in zip(carros, valores): 
    dados = {}
    dados["Preco"] = list(valor.stripped_strings)[0][3:].replace(".","")
    texto = list(carro.stripped_strings)
    dados["Modelo"] = texto[0]
    dados["Ano"] = texto[1].split("/")[0]
    dados["Kilometragem"] = texto[2]
    print(dados)
    print("##################################")
    


{'User-Agent': 'python-requests/2.18.4', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}
{'Preco': '25950', 'Modelo': 'CELTA LT 1.0', 'Ano': '2013', 'Kilometragem': '42.021'}
##################################
{'Preco': '25970', 'Modelo': 'CELTA LT 1.0', 'Ano': '2014', 'Kilometragem': '48.468'}
##################################
{'Preco': '29470', 'Modelo': 'PALIO FIRE 1.0 4P', 'Ano': '2015', 'Kilometragem': '59.556'}
##################################
{'Preco': '29550', 'Modelo': 'NOVO UNO VIVACE', 'Ano': '2014', 'Kilometragem': '24.300'}
##################################
{'Preco': '30090', 'Modelo': 'PALIO FIRE 1.0 4P', 'Ano': '2015', 'Kilometragem': '30.620'}
##################################
{'Preco': '31220', 'Modelo': 'NOVO UNO VIVACE', 'Ano': '2015', 'Kilometragem': '42.584'}
##################################
{'Preco': '31330', 'Modelo': 'GOL 1.6', 'Ano': '2014', 'Kilometragem': '82.435'}
##################################
{'Preco': '31690', '

In [128]:
'1.045.208'.count(".")

2

In [131]:
'1.045.208'.replace(".","",1)

'1045.208'

In [117]:
a = "-"

In [123]:
a = 1.045.208

SyntaxError: invalid syntax (<ipython-input-123-d773e7f4f962>, line 1)

In [ ]:
"rafa".